In [ ]:
from aocd import data, models, submit
from io import StringIO
from pathlib import Path
import re

import pandas as pd
import numpy as np

# Load data and examples

In [ ]:
puzzle_year = 2024
puzzle_day = int(re.match(r"day(\d+)", Path.cwd().name).group(1))

In [ ]:
todays_puzzle = models.Puzzle(year=puzzle_year, day=puzzle_day)
todays_examples = todays_puzzle.examples

# Part A

In [ ]:
def part_a(data: str) -> str:
    digits = [int(digit) for digit in data]
    file_sizes = digits[::2]
    gap_sizes = digits[1::2]
    nr_of_files = len(file_sizes)

    i = file_sizes[0]  # The first file does not contribute to the check sum
    result = 0
    file_index_left = 1
    file_index_right = nr_of_files - 1
    gap_index = 0
    while file_index_left <= file_index_right:
        # we fill the gap corresponding to the indices
        # [i, i+1, i+2, ..., i+len(gap)-1]
        gap_size = gap_sizes[gap_index]
        while gap_size > 0:
            gap_size -= 1
            result += i * file_index_right
            i += 1
            file_sizes[file_index_right] -= 1
            while file_index_right > 1 and file_sizes[file_index_right] < 1:
                file_index_right -= 1
        gap_index += 1
        # now we fill the data file at the indices
        # [i',i'+1,i'+2, ... , i'+len(file)-1]
        # where i'=i+len(gap)-1
        file_size = file_sizes[file_index_left]
        result += file_index_left * (
            i * file_size + (file_size - 1) * file_size // 2
        )  # there is just one file at the current memory position
        i += file_size
        file_sizes[file_index_left] = 0
        file_index_left += 1
    return str(result)

In [ ]:
for example_index, example in enumerate(todays_examples):
    if example.answer_a != "":
        print(
            f"Example {example_index} part a: {part_a(example.input_data)} (expected {example.answer_a})"
        )
        assert part_a(str(example.input_data)) == example.answer_a
submit(part_a(data), part="a", year=puzzle_year, day=puzzle_day)

# Part B

In [ ]:
def part_b(data: str) -> str:
    digits = np.array([int(digit) for digit in data])
    unit_start_location = np.concat([np.array([0]), np.cumsum(digits)[:-1]])
    n_digits = len(digits)

    result = 0
    # We negate the length of the gaps
    digits[1::2] = -digits[1::2]
    unit_index = n_digits
    while unit_index > 0:
        unit_index -= 1
        while digits[unit_index] <= 0:
            unit_index -= 1
        file_size = digits[unit_index]
        file_counter = unit_index // 2
        file_start_memory_position = unit_start_location[unit_index]
        for gap_index in range(1, unit_index, 2):
            # print(f'{gap_index=}, {digits[gap_index]=}')
            if digits[gap_index] <= -1 * file_size:
                # found a place to move the file into
                file_start_memory_position = unit_start_location[gap_index]
                digits[unit_index] = 0
                # fill the gap with the file
                digits[gap_index] += file_size
                unit_start_location[gap_index] += file_size
                break
        file_hash = file_counter * (
            file_start_memory_position * file_size + file_size * (file_size - 1) // 2
        )
        result += file_hash
    return str(result)

In [ ]:
todays_examples[0] = todays_examples[0]._replace(answer_b="2858")